# Question 1: datetime Fundamentals and Time Series Indexing

This question focuses on datetime handling and time series indexing using patient vital signs data.

## Setup

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('default')
sns.set_style('whitegrid')

# Create output directory
os.makedirs('output', exist_ok=True)

## Part 1.1: Load and Explore Data

**Note:** This dataset contains realistic healthcare data characteristics:
- **200 patients** with daily vital signs over 1 year
- **Missing visits**: Patients miss approximately 5% of scheduled visits (realistic!)
- **Different start dates**: Not all patients start monitoring on January 1st (some join later)
- When selecting data by date ranges, you may find that some patients don't have data for certain periods - this is expected and realistic

In [6]:
# Load patient vital signs data
patient_vitals = pd.read_csv('data/patient_vitals.csv')

print("Patient vitals shape:", patient_vitals.shape)
print("Patient vitals columns:", patient_vitals.columns.tolist())

# Display sample data
print("\nPatient vitals sample:")
print(patient_vitals.head())
print("\nData summary:")
print(patient_vitals.describe())

# Check date range and missing data patterns
print(f"\nDate range: {patient_vitals['date'].min()} to {patient_vitals['date'].max()}")
print(f"Unique patients: {patient_vitals['patient_id'].nunique()}")
print(f"Total records: {len(patient_vitals)}")
print(f"Expected records (200 patients × 365 days): {200 * 365:,}")
print(f"Missing visits: ~{200 * 365 - len(patient_vitals):,} records")

Patient vitals shape: (63624, 7)
Patient vitals columns: ['date', 'patient_id', 'temperature', 'heart_rate', 'blood_pressure_systolic', 'blood_pressure_diastolic', 'weight']

Patient vitals sample:
         date patient_id  temperature  heart_rate  blood_pressure_systolic  \
0  2023-02-22      P0001         98.4   84.018315                      125   
1  2023-02-23      P0001         97.7   81.152032                      131   
2  2023-02-24      P0001         98.8   88.550207                      131   
3  2023-02-26      P0001         99.0   87.471654                      127   
4  2023-02-27      P0001         98.3   84.540929                      130   

   blood_pressure_diastolic  weight  
0                        83    68.9  
1                        89    69.0  
2                        89    67.8  
3                        85    69.6  
4                        88    68.2  

Data summary:
        temperature    heart_rate  blood_pressure_systolic  \
count  63624.000000  63624.0

## Part 1.2: datetime Operations

**TODO: Perform datetime operations**

In [7]:
# Load data
patient_vitals = pd.read_csv('data/patient_vitals.csv')

# TODO: Convert date column to datetime
patient_vitals['date'] = pd.to_datetime(patient_vitals['date'])
# Extract year, month, day components
patient_vitals['year'] = patient_vitals['date'].dt.year
patient_vitals['month'] = patient_vitals['date'].dt.month
patient_vitals['day'] = patient_vitals['date'].dt.day

# TODO: Calculate time differences (e.g., days since first measurement)
# Note: Since patients start at different times, calculate days_since_start per patient
# Hint: To use groupby on the 'date' column, temporarily reset the index, then set it back
# Example: patient_vitals_reset = patient_vitals.reset_index()
#          Use groupby('patient_id')['date'].transform(lambda x: (x - x.min()).dt.days)
#          Or use groupby('patient_id').apply() to calculate days from each patient's first date
#          Then: patient_vitals = patient_vitals_reset.set_index('date')
# patient_vitals['days_since_start'] = None  # Calculate from each patient's start date
patient_vitals = patient_vitals.reset_index()
patient_vitals['days_since_start'] = patient_vitals.groupby('patient_id')['date'].transform(lambda x: (x - x.min()).dt.days)


# TODO: Create business day ranges for clinic visit schedules
clinic_dates = pd.date_range(start=patient_vitals['date'].min(), end=patient_vitals['date'].max(), freq='B')

# TODO: Create date ranges with different frequencies
# Daily monitoring schedule
daily_range = pd.date_range(start=patient_vitals['date'].min(), end=patient_vitals['date'].max(), freq='D')
# Weekly lab test schedule (Mondays)
weekly_range = pd.date_range(start=patient_vitals['date'].min(), end=patient_vitals['date'].max(), freq='W-MON')
# Monthly checkup schedule
monthly_range = pd.date_range(start=patient_vitals['date'].min(), end=patient_vitals['date'].max(), freq='MS') 

# TODO: Use date ranges to analyze visit patterns
# Check how many patient visits occurred on clinic business days vs weekends
patient_dates_set = set(patient_vitals['date'].dt.date)
clinic_dates_set = set(clinic_dates.date)
visits_on_clinic_days = len(patient_dates_set & clinic_dates_set)
visits_on_weekends = len(patient_dates_set) - visits_on_clinic_days
print(f"Visits on clinic business days: {visits_on_clinic_days}")
print(f"Visits on weekends: {visits_on_weekends}")
print(f"Total unique visit dates: {len(patient_dates_set)}")

# TODO: Save results as 'output/q1_datetime_analysis.csv'
# Create a DataFrame with datetime analysis results including:
# - date (datetime index or column)
# - year, month, day (extracted from datetime)
# - days_since_start (calculated time differences)
# - patient_id
# - At least one original column (e.g., temperature, heart_rate)
# Note: When saving to CSV with index=False, you'll need to convert the index to a column first
# Example structure:
datetime_analysis = patient_vitals[['date','patient_id', 'year', 'month', 'day', 'days_since_start', 'temperature', 'heart_rate']]

datetime_analysis.to_csv('output/q1_datetime_analysis.csv', index=False)
print("q1_datetime_analysis.csv saved with columns:", datetime_analysis.columns.tolist())

Visits on clinic business days: 260
Visits on weekends: 105
Total unique visit dates: 365
q1_datetime_analysis.csv saved with columns: ['date', 'patient_id', 'year', 'month', 'day', 'days_since_start', 'temperature', 'heart_rate']


## Part 1.3: Time Zone Handling

**TODO: Handle time zones**

In [8]:
# TODO: Create timezone-aware datetime (for multi-site clinical trials)
  # Current time in UTC
utc_time = pd.Timestamp.utcnow()
# Convert to US Eastern
eastern_time = utc_time.tz_convert('US/Eastern')
print("Current UTC Time:", utc_time)
print("Current Eastern Time:", eastern_time)

# TODO: Convert between different timezones
# Create timezone-aware DataFrame from patient_vitals
# Localize to UTC
patient_vitals_tz = patient_vitals.copy()
patient_vitals_tz['date'] = pd.to_datetime(patient_vitals_tz['date'])
patient_vitals_tz = patient_vitals_tz.set_index('date')
patient_vitals_tz.index = patient_vitals_tz.index.tz_localize('UTC')
print("\nUTC Timezone-aware datetimes:")
print(patient_vitals_tz.head())

# Convert to Eastern time
patient_vitals_tz_eastern = patient_vitals_tz.copy()
patient_vitals_tz_eastern.index = patient_vitals_tz_eastern.index.tz_convert('US/Eastern')
print("\nConverted to US/Eastern timezone:")
print(patient_vitals_tz_eastern.head())

# TODO: Handle daylight saving time transitions
# Create datetime that spans DST transition
# Note: Using UTC avoids DST ambiguity issues - UTC has no daylight saving time
# Best practice: Store data in UTC, convert to local timezones only when needed
# UTC time avoids DST issues
dst_date_utc = pd.Timestamp('2023-03-12 10:00:00', tz='UTC')
# Convert UTC to Eastern
dst_time_eastern = dst_date_utc.tz_convert('US/Eastern')
print(f"DST transition example - UTC: {dst_date_utc}, Eastern: {dst_time_eastern}")

# TODO: Document timezone operations
# Create a report string with the following sections:
# 1. Original timezone: Describe what timezone your original data was in (or if it was naive)
# 2. Localization method: Explain how you localized the data (e.g., tz_localize('UTC'))
# 3. Conversion: Describe what timezone you converted to (e.g., 'US/Eastern')
# 4. DST handling: Document any issues or observations about daylight saving time transitions
#    Note: Explain why using UTC as the base timezone avoids DST ambiguity issues
# 5. Example: Show at least one example of a datetime before and after conversion
# Minimum length: 200 words
timezone_report = """Timezone Operations Report

TODO: Document your timezone operations:
- The original data was in naive datetime format (no timezone info).
- To localize the data, I used tz_localize('UTC') to assign UTC timezone to the naive datetimes.
- The UTC-localized datetimes were then converted to 'US/Eastern' timezone using tz_convert('US/Eastern').
- UTC was chosen as the base timezone to avoid issues with daylight saving time (DST) transitions, which can create ambiguity in local times. 
By storing data in UTC, we ensure that all timestamps are consistent and unambiguous, as UTC does not observe DST.
- An example of a datetime before and after conversion:
  - Original UTC datetime: 2023-03-12 10:00:00+00:00
  - Converted Eastern datetime: 2023-03-12 05:00:00-05:00
- UTC is recommended for storing timestamps in multi-site clinical trials to maintain consistency across different locations and timezones.
Storing timestamps in UTC ensures calculations, comparisons, and merging across sites are unambiguous.
"""

# TODO: Save results as 'output/q1_timezone_report.txt'
with open('output/q1_timezone_report.txt', 'w') as f:
    f.write(timezone_report)

print("Timezone report saved to q1_timezone_report.txt")

Current UTC Time: 2025-11-20 02:31:15.839070+00:00
Current Eastern Time: 2025-11-19 21:31:15.839070-05:00

UTC Timezone-aware datetimes:
                           index patient_id  temperature  heart_rate  \
date                                                                   
2023-02-22 00:00:00+00:00      0      P0001         98.4   84.018315   
2023-02-23 00:00:00+00:00      1      P0001         97.7   81.152032   
2023-02-24 00:00:00+00:00      2      P0001         98.8   88.550207   
2023-02-26 00:00:00+00:00      3      P0001         99.0   87.471654   
2023-02-27 00:00:00+00:00      4      P0001         98.3   84.540929   

                           blood_pressure_systolic  blood_pressure_diastolic  \
date                                                                           
2023-02-22 00:00:00+00:00                      125                        83   
2023-02-23 00:00:00+00:00                      131                        89   
2023-02-24 00:00:00+00:00             

## Submission Checklist

Before moving to Question 2, verify you've created:

- [ ] `output/q1_datetime_analysis.csv` - datetime analysis results
- [ ] `output/q1_timezone_report.txt` - timezone handling report
